# AFSK Demodulator
## Step 5: Hardware I/O Interlude

This is a Pynq portion of the AFSK demodulator project. We will be using the FPGA overlay that we
created in Vivado. At this point we have created the bitstream for "count" and copied the
bitstream, TCL wrapper, and hardware hand-off file to the Pynq board.

This module is designed to exercise the digital I/O of the board, both inputs and outputs.  The PL implements a 4-bit counter using the green LEDs, and an up/down input using buttons 2 & 1, a reset to 0 on button 0, and a "read me" button that triggers an interrupt on the PS on button 3.  The 4-bit counter can be read on the PS via MMIO.

Let's first verify that we can load the module.

In [7]:
from pynq import Overlay, Xlnk
import pynq.lib.dma

overlay = Overlay('count.bit')
count = overlay.count
print(count.register_map)

RegisterMap {
  cntr_V = Register(cntr_V=8, RESERVED=0),
  cntr_V_ctrl = Register(cntr_V_ap_vld=1, RESERVED=0)
}


In the following section we will implement code that waits for an interrupt.  This is triggered by pressing button 3 (the left-most button).  When it is pressed, the value of the counter is read and printed below.  If the value is 0 then the loop terminates.  To restart, just re-run the cell.

One thing that can be rather unexpected is that when `cntr` is defined as an `int` in HLS, it's name in Python is `cntr`.  However, if it is defined as `ap_uint<4>` its name is `cntr_V`.  There does not appear to be any way to affect this naming.  Hungarian notation in software design is now universally discouraged.

In [8]:
import asyncio

async def print_count():
    while True:
        await count.ap_return.wait()
        a = count.register_map.cntr_V
        print(a)
        count.ap_return.event.clear()
        if int(a) == 0:
            return

loop = asyncio.get_event_loop()
loop.run_until_complete(print_count())

0x8
0x7
0x0
